In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)





In [4]:
df_train = pd.read_csv('train.csv', index_col = 'id')
df_train.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [5]:
X = df_train.drop('species', axis=1)
y = df_train.species
df_train.shape, X.shape, y.shape

((990, 193), (990, 192), (990L,))

In [6]:
import sklearn.preprocessing as skpp
classes = df_train.species.unique()
classes.sort()

y = skpp.label_binarize(y, classes = classes)
y[0,]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [9]:
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as skpp
import sklearn.decomposition as skdc
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

norm = skpp.StandardScaler()
pca = skdc.PCA(n_components=25) 
svm = OneVsRestClassifier(SVC(kernel='linear', probability=True))
pipe = Pipeline(steps = [
        ('standardizer', norm), 
        ('decom', pca), 
        ('alg', svm)])

In [11]:
import sklearn.model_selection as skms

params = [
    {
        'decom__n_components': (25, 50),
        'alg__estimator__C': list(np.arange(.5, 1.5, .5)),
        'alg__estimator__kernel': ['linear']
    }
]

grid = skms.GridSearchCV(pipe, params)

In [12]:
import sklearn.model_selection as skms

strat_cv_shuffler = skms.StratifiedShuffleSplit(n_splits = 2, train_size=0.7)

In [13]:
scores = skms.cross_val_score(grid, X, y, cv=strat_cv_shuffler)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 71 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 74 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 68 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 95 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 43 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 83 is present in all training examples.
  str(classes[c]))
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 97 is present in all tr

Accuracy: 0.91 (+/- 0.06)


In [14]:
grid.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('standardizer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('decom', PCA(copy=True, iterated_power='auto', n_components=25, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('alg', OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, cla...ability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'decom__n_components': (25, 50), 'alg__estimator__kernel': ['linear'], 'alg__estimator__C': [0.5, 1.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [16]:
df_test = pd.read_csv('test.csv', index_col = 'id')

pred = grid.predict_proba(df_test)
df_sub = pd.DataFrame(pred, index = df_test.index, columns = classes)
df_sub.to_csv('submission.csv')
df_sub.head()

,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
id,,,,,,,,,,,,,,,,,,,,,
4,0.000031,0.000320,3.951411e-07,2.312671e-02,0.000488,8.590742e-06,0.000001,0.000002,1.033965e-07,0.000205,...,0.000006,0.000835,0.000029,0.000004,0.000003,0.000032,0.000014,1.000000e-07,0.000278,5.203584e-07
7,0.000015,0.000092,8.246265e-05,1.300150e-03,0.000516,1.746719e-05,0.020413,0.000015,1.108483e-05,0.000286,...,0.000368,0.000914,0.000041,0.000011,0.000002,0.016724,0.000500,2.488186e-03,0.000038,1.349563e-04
9,0.000459,0.768644,5.722815e-04,1.163087e-05,0.027046,1.155422e-04,0.000124,0.021486,5.018152e-03,0.001995,...,0.001949,0.000655,0.000193,0.000055,0.000005,0.000027,0.000174,2.997928e-06,0.000127,4.015411e-03
12,0.000015,0.009180,2.064534e-03,1.373078e-07,0.000449,3.595095e-07,0.000962,0.000177,4.511491e-03,0.002183,...,0.002003,0.000226,0.000334,0.000011,0.000036,0.001414,0.022767,9.662506e-05,0.000311,1.232842e-02
13,0.001886,0.000567,8.941147e-05,1.000000e-07,0.000403,1.000000e-07,0.000904,0.002747,4.740577e-03,0.002271,...,0.002188,0.000086,0.008597,0.000040,0.002555,0.002198,0.037173,1.520356e-04,0.000855,7.902532e-05
